In [1]:
import argparse
import numpy as np
import os
import sys
import tensorflow as tf
import yaml
from yaml import CLoader as Loader

from lib.utils import load_graph_data
from model.dcrnn_supervisor import DCRNNSupervisor

sys.path.append(os.getcwd())
parser = argparse.ArgumentParser()
parser.add_argument('--use_cpu_only', default=False, type=str, help='Whether to run tensorflow on cpu.')
parser.add_argument('--config_filename', default='data/model/pretrained/METR-LA/config.yaml', type=str,
                    help='Config file for pretrained model.')
parser.add_argument('--output_filename', default='data/dcrnn_predictions.npz')
parser.add_argument('-f', default=None, type=str)
args = parser.parse_args()


with open(args.config_filename) as f:
    config = yaml.load(f, Loader)

tf_config = tf.ConfigProto()
if args.use_cpu_only:
    tf_config = tf.ConfigProto(device_count={'GPU': 0})
tf_config.gpu_options.allow_growth = True
graph_pkl_filename = config['data']['graph_pkl_filename']
_, _, adj_mx = load_graph_data(graph_pkl_filename)
with tf.Session(config=tf_config) as sess:
    supervisor = DCRNNSupervisor(adj_mx=adj_mx, **config)
    supervisor.load(sess, config['train']['model_filename'])
    outputs = supervisor.evaluate(sess)
    np.savez_compressed(args.output_filename, **outputs)
    print('Predictions saved as {}.'.format(args.output_filename))



2022-02-13 18:24:12,431 - INFO - Log directory: data/model/pretrained/METR-LA

2022-02-13 18:24:12,432 - INFO - {'base_dir': 'data/model', 'log_level': 'INFO', 'data': {'batch_size': 64, 'dataset_dir': 'data/METR-LA', 'graph_pkl_filename': 'data/sensor_graph/adj_mx.pkl', 'test_batch_size': 64}, 'model': {'cl_decay_steps': 2000, 'filter_type': 'dual_random_walk', 'horizon': 12, 'input_dim': 2, 'l1_decay': 0, 'max_diffusion_step': 2, 'num_nodes': 207, 'num_rnn_layers': 2, 'output_dim': 1, 'rnn_units': 64, 'seq_len': 12, 'use_curriculum_learning': True}, 'train': {'base_lr': 0.01, 'dropout': 0, 'epoch': 64, 'epochs': 100, 'epsilon': 0.001, 'global_step': 24375, 'log_dir': 'data/model/pretrained/METR-LA', 'lr_decay_ratio': 0.1, 'max_grad_norm': 5, 'max_to_keep': 100, 'min_learning_rate': 2e-06, 'model_filename': 'data/model/pretrained/METR-LA/models-2.7422-24375', 'optimizer': 'adam', 'patience': 50, 'steps': [20, 30, 40, 50], 'test_every_n_epochs': 10}}


2022-02-13 18:24:12.404175: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-02-13 18:24:12.427254: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3693290000 Hz
2022-02-13 18:24:12.429421: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x560da3997d80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-02-13 18:24:12.429440: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


2022-02-13 18:24:19,068 - INFO - ('x_train', (23974, 12, 207, 2))
2022-02-13 18:24:19,068 - INFO - ('y_train', (23974, 12, 207, 2))
2022-02-13 18:24:19,069 - INFO - ('x_val', (3425, 12, 207, 2))
2022-02-13 18:24:19,069 - INFO - ('y_val', (3425, 12, 207, 2))
2022-02-13 18:24:19,069 - INFO - ('x_test', (6850, 12, 207, 2))
2022-02-13 18:24:19,070 - INFO - ('y_test', (6850, 12, 207, 2))



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API

